In [9]:
import os
import sys
import time
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import collections
import random

from cloudvolume import CloudVolume
from cloudvolume.lib import Vec, Bbox
from skimage.transform import resize

def invert_mask(mask, th=50/255.0, bias=0.2):
    bitmask = mask < th
    th_mask = bitmask * mask
    th_max = np.max(th_mask) + bias
    return (th_max - th_mask) / th_max

class Stack():
  """Collection of CloudVolume paths, with load & save operations
  """

  def __init__(self, paths):
    self.paths = paths

  def load(self, key, bbox, src_mip=0, dst_mip=0):
    src_cv = CloudVolume(self.paths[key], mip=src_mip, fill_missing=True)
    src_bbox = Bbox.from_slices(src_cv.slices_from_global_coords(bbox))
    img = src_cv[src_bbox.to_slices()][:,:,0,0]
    if dst_mip != src_mip:
        dst_cv = CloudVolume(self.paths[key], mip=dst_mip, fill_missing=True)
        dst_bbox = Bbox.from_slices(dst_cv.slices_from_global_coords(bbox))
        img = resize(img, dst_bbox.size3()[:2], preserve_range=True)
    return img

  def save(self, key, mip, bbox, img, is_field=False):
    """Save vector field to CloudVolume
    
    Args:
    * cv: CloudVolume object
    * mip: int representing mip-level of the CloudVolume
    * bbox: Bbox at mip 0 with size3 of (x,y,1)
    * img: nxm int64 ndarray with shape (x,y)
    * is_field: boolean whether img is a field object
        field object is 4d tensor with shape (1,x,y,2)
    """
    src_cv = CloudVolume(self.paths['image'], mip=mip)
    info = src_cv.info
    if is_field:
        info['data_type'] = 'float32'
        info['num_channels'] = 2
    cv = CloudVolume(self.paths[key], mip=mip, bounded=False, 
                     info=info, fill_missing=True, 
                     non_aligned_writes=True, cdn_cache=False)
    cv.commit_info()
    
    if is_field:
        s = cv.slices_from_global_coords(bbox) 
        s += (slice(0,2),)
        cv[s] = img.permute(1,2,0,3).data.cpu().numpy()
    else:
        s = cv.slices_from_global_coords(bbox)
        cv[s] = img[:,:,np.newaxis,np.newaxis]

class Optimizer():
  """
  
  Attributes:
    * src_image: nxm float64 ndarry normalized between [0,1]
      This is the image to be transformed by the returned vector field.
    * dst_images: list of nxm float64 ndarrays normalized between [0,1]
      This is the set of images that transformed src_image will be compared to.
    * src_mask_smooth: nxm float64 ndarray normalized between [0,1]
      The weight represents that degree to which a pixel participates in smooth
      deformation (0: not at all; 1: completely). This mask is used to reduce 
      the smoothness penalty for pixels that participate in a non-smooth 
      deformation.
    * src_mask_loss: nxm float64 ndarray normalized between [0,1]
      This mask is transformed by the vector field and used to reduce the MSE
      for pixels that participate in a defect.
    * dst_masks_loss: list of nxm float64 ndarrays normalized between [0,1]
      Exactly like the src_mask_loss above. These masks are only used to reduce the 
      MSE for pixels that participate in a defect.
  """
    
  def __init__(self, stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip):
    self.stack = stack
    self.bbox = bbox
    self.image_mip = image_mip
    self.output_mip = output_mip
    self.src_image = stack.load('image', bbox, src_mip=image_mip, dst_mip=image_mip) / 255.0
    self.src_fold = stack.load('fold', bbox, src_mip=fold_mip, dst_mip=image_mip) / 255.0
    self.src_crack = stack.load('crack', bbox, src_mip=crack_mip, dst_mip=image_mip) / 255.0
    self.src_smooth_mask = invert_mask(np.minimum(self.src_fold, self.src_crack))
    self.src_loss_mask = (stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
    
    self.dst_images = []
    self.dst_masks = []
    for z in dst_increment:
        dst_bbox = self.bbox + [0,0,z]
        dst_image = stack.load('image', dst_bbox, src_mip=image_mip, dst_mip=image_mip)
        self.dst_images.append(dst_image)
        dst_mask = (stack.load('loss', dst_bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
        self.dst_masks.append(dst_mask)
        if output_mip != image_mip:
            dst_image = stack.load('image', dst_bbox, src_mip=output_mip, dst_mip=output_mip)
        self.stack.save('output', output_mip, dst_bbox, dst_image, is_field=False)
    
    self.identities = {}
        
  @staticmethod
  def center(var, dims, d):
      if not isinstance(d, collections.Sequence):
          d = [d for i in range(len(dims))]
      for idx, dim in enumerate(dims):
          if d[idx] == 0:
              continue
          var = var.narrow(dim, int(d[idx]/2), int(var.size()[dim] - d[idx]))
      return var

  def get_identity_grid(self, dim, cache=True):
      if dim not in self.identities:
          gx, gy = np.linspace(-1, 1, dim), np.linspace(-1, 1, dim)
          I = np.stack(np.meshgrid(gx, gy))
          I = np.expand_dims(I, 0)
          I = torch.FloatTensor(I)
          I = torch.autograd.Variable(I, requires_grad=False)
          I = I.permute(0,2,3,1).cuda()
          self.identities[dim] = I
      if cache:
          return self.identities[dim]
      else:
          return self.identities[dim].clone()

  def jacob(self, fields):
      def dx(f):
          p = Variable(torch.zeros((1,1,f.size(1),2))).cuda()
          return torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
      def dy(f):
          p = Variable(torch.zeros((1,f.size(1),1,2))).cuda()
          return torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
      fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
      field = torch.sum(torch.cat(fields, -1) ** 2, -1)
      return field

  def penalty(self, fields, mask=1):
      jacob = self.jacob(fields)
      jacob = torch.mul(jacob, mask)
      return torch.sum(jacob)

  def render(self, field):
      src = stack.load('image', bbox, src_mip=self.output_mip, dst_mip=self.output_mip) / 255.0
      src = torch.FloatTensor(src).cuda()
      src = Variable(src).unsqueeze(0).unsqueeze(0)
      if self.output_mip < self.image_mip:
          upsample = lambda x: nn.Upsample(scale_factor=2**x, mode='bilinear') if x >0 else (lambda y: y)
          field = upsample(self.image_mip - self.output_mip)(field.permute(0,3,1,2)).permute(0,2,3,1)
      y = F.grid_sample(src, field + self.get_identity_grid(field.size(2)))
      img = y.data.cpu().numpy()
      img = (img[0,0,:,:]*255).astype(np.uint8)
      self.stack.save('output', self.output_mip, self.bbox, img, is_field=False)

  def process(self, field=None, crop=0, write_field=True, ndownsamples=4, 
                                   currn=5, avgn=20, lambda1=0.4, lr=0.2, 
                                   eps=0.01, min_iter=20, max_iter=1000):
    """Compute vector field that minimizes mean squared error (MSE) between 
    transformed src_image & all dst_images regularized by the smoothness of the 
    vector field subject to masks that allow the vector field to not be smooth. 
    The minimization uses SGD.

    Args:
    * field: nxmx2 float64 torch tensor normalized between [0,1]
      This field represents the derived vector field that transforms the 
      src_image subject to the contraints of the minimization.
    
    Returns:
    * field: nxmx2 float64 torch tensor normalized between [0,1]
    """
    downsample = lambda x: nn.AvgPool2d(2**x,2**x, count_include_pad=False) if x > 0 else (lambda y: y)
    upsample = nn.Upsample(scale_factor=2, mode='bilinear')
    s = torch.FloatTensor(self.src_image)
    src_var = Variable((s - torch.mean(s)) / torch.std(s)).cuda().unsqueeze(0).unsqueeze(0)
    src_smooth_mask_var = Variable(torch.FloatTensor(self.src_smooth_mask)).cuda().unsqueeze(0).unsqueeze(0)
    src_loss_mask_var = Variable(torch.FloatTensor(self.src_loss_mask)).cuda().unsqueeze(0).unsqueeze(0)
    dst_vars = []
    dst_mask_vars = []
    for d, m in zip(self.dst_images, self.dst_masks):
      d = torch.FloatTensor(d)
      dst_var = Variable((d - torch.mean(d)) / torch.std(d)).cuda().unsqueeze(0).unsqueeze(0)
      dst_vars.append(dst_var)
      mask_var = Variable(torch.FloatTensor(m)).cuda().unsqueeze(0).unsqueeze(0)
      dst_mask_vars.append(mask_var)
      
    dim = int(src_var.size()[-1] / (2 ** (ndownsamples - 1)))
    if field is None:
        field = Variable(torch.zeros((1,dim,dim,2))).cuda().detach()
    else:
        field = Variable(field).cuda().detach()
    field.requires_grad = True
    updates = 0
    for k in reversed(range(ndownsamples)):
      src_ = downsample(k)(src_var).detach()
      src_.requires_grad = False
      src_smooth_mask_ = downsample(k)(src_smooth_mask_var).detach()
      src_smooth_mask_.requires_grad = False
      src_loss_mask_ = downsample(k)(src_loss_mask_var).detach()
      src_loss_mask_.requires_grad = False
      dst_list_, dst_mask_list_ = [], []
      for d, m in zip(dst_vars, dst_mask_vars):
        dst_ = downsample(k)(d).detach()
        dst_.requires_grad = False
        dst_list_.append(dst_)
        mask_ = downsample(k)(m).detach()
        mask_.requires_grad = False
        dst_mask_list_.append(mask_)
      field = field.detach()
      field.requires_grad = True
      opt = torch.optim.SGD([field], lr=lr/(k+1))
      #sched = lr_scheduler.StepLR(opt, step_size=1, gamma=0.995)
      costs = []
      start_updates = updates
      print('Downsample factor {0}x'.format(2**k))
      while True:
        updates += 1
        I = self.get_identity_grid(field.size(2))
        pred = F.grid_sample(src_, field + I)
        pred_loss_mask = F.grid_sample(src_loss_mask_, field + I)
        pred_smooth_mask = F.grid_sample(src_smooth_mask_, field + I)
        centered_mask = self.center(pred_smooth_mask.squeeze(0), (1,2), 128/(2**k))
        centered_field = self.center(field, (1,2), 128/(2**k))
        penalty1 = self.penalty([centered_field], centered_mask)
        cost = penalty1 * lambda1/(k+1)
        for d, m in zip(dst_list_, dst_mask_list_):
          mask = torch.mul(pred_loss_mask, m)
          mse = torch.mul(pred - d, mask)**2
          cost += torch.mean(self.center(mse, (-1,-2), 128 / (2**k)))
        #cost = diff + penalty1 * lambda1/(k+1)
        print(cost.data.cpu().numpy())
        costs.append(cost)
        cost.backward()
        opt.step()
        #sched.step()uniform
        opt.zero_grad()
        if len(costs) > avgn + currn and len(costs)>min_iter:
            hist_costs = costs[-(avgn+currn):-currn]
            hist = sum(hist_costs).data[0] / avgn
            curr = sum(costs[-currn:]).data[0] / currn
            if abs((hist-curr)/hist) < eps/(2**k) or len(costs)>max_iter:
                break
        #print downsamples, updates - start_updates
      if write_field:
        self.stack.save('field', self.image_mip+k, self.bbox, field + I, is_field=True)
      if k > 0:
        field = upsample(field.permute(0,3,1,2)).permute(0,2,3,1)
    #print(cost.data[0], diff.data[0], penalty1.data[0])
    print('done:', updates)
    print(field.shape)
    # return self.center(field, (1,2), crop*2).data.cpu().numpy()[0]
    return self.center(field, (1,2), crop*2)

In [12]:
name = 'refactor_invert_smoothness_v1'
paths = {'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
         'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
         'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
         'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
         'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/{0}'.format(name),
         'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/{0}/field'.format(name)}
stack = Stack(paths)
offsets = [Vec(102716, 107077, 526), # fold
           Vec(102320, 111106, 526), # crack
           Vec(102565, 103904, 527)] # no defect
size = Vec(2048, 2048, 1)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]
for offset in offsets:
    bbox = Bbox(offset, offset+size)
    o = Optimizer(stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip)
    field = o.process(ndownsamples=5, currn=5, avgn=20, lambda1=0.9, lr=0.1, 
                                                  eps=0.001, min_iter=100, max_iter=5000)
    o.render(field)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.94it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.06it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.19it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.81it/s]

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?

Downsample factor 16x
[0.26509356]
[0.2521733]
[0.24393186]
[0.23815098]
[0.23347685]
[0.22946411]
[0.22585744]
[0.22247913]
[0.21925554]
[0.2162427]
[0.21328422]
[0.21043533]
[0.20770875]
[0.20501432]
[0.20245807]
[0.1999239]
[0.19751918]
[0.19514284]
[0.19290888]
[0.19073072]
[0.18860883]
[0.18658316]
[0.18465322]
[0.18275858]
[0.18098465]
[0.1793024]
[0.17764139]
[0.17609398]
[0.17459255]
[0.17320535]
[0.17184502]
[0.17055333]
[0.16931531]
[0.16816595]
[0.16703036]
[0.1659919]
[0.16496935]
[0.16396132]
[0.16308862]
[0.16216423]
[0.1613245]
[0.16052598]
[0.15972733]
[0.15893821]
[0.15827839]
[0.15763143]
[0.15690523]
[0.15631697]
[0.15570079]
[0.1550788]
[0.15455367]
[0.1539746]
[0.15341371]
[0.15294841]
[0.15239766]
[0.15195483]
[0.15146413]
[0.15099153]
[0.15059268]
[0.1501267]
[0.14973721]
[0.14933713]
[0.14895087]
[0.14856559]
[0.14822164]
[0.14786175]
[0.14751811]
[0.14720449]
[0.14687008]
[0.14660645]
[0.14627615]
[0.1460087]
[0.1456941]
[0.14548714]
[0.14518046]
[0.14494967]
[

[0.13156709]
[0.13155724]
[0.13152045]
[0.13155541]
[0.13153756]
[0.13154742]
[0.13153455]
[0.13154262]
[0.13152173]
[0.13153479]
[0.1315524]
[0.13150993]
[0.13153604]
[0.13153648]
[0.13150316]
[0.13155231]
[0.13151765]
[0.13149756]
[0.13154465]
[0.13150434]
[0.13153023]
[0.13151813]
[0.13149206]
[0.13151559]
[0.13154674]
[0.13148388]
[0.1315192]
[0.13149926]
[0.13150804]
[0.13151376]
[0.13149762]
[0.13151512]
[0.13148251]
[0.13148491]
[0.1315385]
[0.13148016]
[0.1315001]
[0.13149229]
[0.13148221]
[0.13151085]
[0.13150212]
[0.13146967]
[0.13150439]
[0.13147072]
[0.13149096]
[0.1314941]
[0.1315014]
[0.13143972]
[0.13151254]
[0.13147552]
[0.1314882]
[0.13146466]
[0.13149756]


Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.28it/s]



Downsample factor 8x
[0.5632943]
[0.5405192]
[0.5210751]
[0.5037061]
[0.48821157]
[0.47439277]
[0.46254694]
[0.45207274]
[0.44326097]
[0.43633467]
[0.4291478]
[0.42296323]
[0.4175542]
[0.41267607]
[0.40885347]
[0.4047071]
[0.4013434]
[0.3983346]
[0.3952595]
[0.3921209]
[0.3901028]
[0.38788986]
[0.38531637]
[0.38414586]
[0.38226444]
[0.38029215]
[0.3798085]
[0.37799862]
[0.37660658]
[0.37608382]
[0.37447464]
[0.37334573]
[0.3726104]
[0.37159294]
[0.37079322]
[0.3700227]
[0.369347]
[0.36880848]
[0.36792746]
[0.36760294]
[0.36700952]
[0.36611852]
[0.36598054]
[0.3655548]
[0.3648401]
[0.36441952]
[0.36427557]
[0.3637008]
[0.36312503]
[0.36309576]
[0.3627143]
[0.3620555]
[0.36210358]
[0.36173338]
[0.36093467]
[0.3608565]
[0.36089694]
[0.36005768]
[0.35996547]
[0.36004406]
[0.3592083]
[0.35903263]
[0.3593345]
[0.3585531]
[0.35824054]
[0.3585288]
[0.35793024]
[0.35760236]
[0.3578617]
[0.3573319]
[0.3570759]
[0.35712415]
[0.3570077]
[0.3566781]
[0.35658503]
[0.3564217]
[0.35626832]
[0.35606927

...

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  5.76it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  5.00it/s]



Downsample factor 1x
[1.6181979]
[1.2623601]
[1.2179238]
[1.2058694]
[1.2000679]
[1.1962316]
[1.1934326]
[1.1911451]
[1.1892763]
[1.1876388]
[1.1862452]
[1.1850113]
[1.1838806]
[1.1828716]
[1.1819577]
[1.1811907]
[1.1803899]
[1.1797078]
[1.178951]
[1.1783576]
[1.1777158]
[1.1771908]
[1.1766584]
[1.1762707]
[1.1757425]
[1.1753886]
[1.1749661]
[1.1745865]
[1.174166]
[1.1738918]
[1.1734991]
[1.1732042]
[1.1728301]
[1.1726369]
[1.1722807]
[1.1720436]
[1.1717033]
[1.1715422]
[1.1712213]
[1.171031]
[1.170784]
[1.1706152]
[1.1703286]
[1.1701915]
[1.1699705]
[1.1698155]
[1.1696057]
[1.1694808]
[1.1692153]
[1.1691275]
[1.1689091]
[1.168814]
[1.1686124]
[1.168525]
[1.1683472]
[1.1682475]
[1.1680164]
[1.167917]
[1.1677495]
[1.1676686]
[1.1675079]
[1.1674196]
[1.16727]
[1.1672212]
[1.1670594]
[1.1670239]
[1.1668328]
[1.1667728]
[1.1666319]
[1.1665859]
[1.1664691]
[1.1663506]
[1.1662177]
[1.166188]
[1.1660359]
[1.1659997]
[1.1658788]
[1.1657836]
[1.1656823]
[1.165653]
[1.1655526]
[1.1655166]
[1.165

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.96it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.69it/s]

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]

done: 1415
torch.Size([1, 512, 512, 2])



Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9.02it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.14it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.39it/s]

